In [1]:
import datetime
import seaborn as sns
import pymysql
import pandas as pd
import math
import numpy as np

import re
con = pymysql.connect(host='localhost', user='root',password='Karelia',database= 'w3')


In [363]:
def custom_query(command,joins,wheres=False,havings=False):
    #query constructed from table names, joins as attributed by Db_command class
    con.connect()
    select_main = con.cursor()
    statement = 'select {} from orders {}'.format(command,joins)
    if wheres or havings:
        end_statement = " and orders.OrderID IN ".join([*query.wheres,*query.havings]) 
        statement = statement +' where orders.OrderID IN ' + end_statement
    select_main.execute(statement)
    field_names = [i[0] for i in select_main.description]
    rows = select_main.fetchall()
    con.commit()
    con.close()
    sales = pd.DataFrame(rows,columns=field_names)
    sales = sales.loc[:,~sales.columns.duplicated()]
    sales['OrderID'] = sales['OrderID'].astype(str)
    return sales

In [398]:
class Db_command:
    #a class structure for creating an sql query depending on the requests column names
    def __init__(query,keys=False,command=False,joins=False,wheres=False,havings=False):
        #get the table names and column names from database
        con.connect()
        select_main = con.cursor()
        select_main.callproc('db_rels')
        rows = select_main.fetchall()
        con.commit()
        con.close()
        keys = {i[1]:{"command":i[0]+"."+ i[1],"link":i[0]} for i in rows}

        del keys['Country']
        del keys['iso_code']
        del keys['City']
        del keys['point_id']

        #create custom keys for aliases in our key
        keys['Total'] = {"command":"products.Price * order_details.Quantity as 'Total'","link":"products"}
        keys['SalesPerson'] = {"command":"concat( employees.FirstName,' ',employees.LastName) as 'SalesPerson'","link":"employees"}
        keys['CustomerCity'] = {"command":"customers.City as 'CustomerCity'",'link':'customers'}
        keys['SupplierCity'] = {"command":"suppliers.City as 'SupplierCity'",'link':'suppliers'}
        keys['CustomerCountry'] = {"command":"customers.Country as 'CustomerCountry'",'link':'customers'} 
        keys['SupplierCountry'] =  {'command':"suppliers.Country as 'SupplierCountry'",'link':'suppliers'}
        keys['customer_iso'] ={"command": "customer_iso_ref.iso_code as 'customer_iso'","link":"customer_iso"}
        keys['supplier_iso'] = {"command":"supplier_iso_ref.iso_code as 'supplier_iso'","link":"supplier_iso"}
        keys['customer_point'] = {"command":"customers.City as 'customer_point',customer_point_ref.latitude as 'customer_lat',customer_point_ref.longitude as 'customer_lon'",'link':'customer_point'}
        keys['supplier_point'] = {"command":"suppliers.City as 'supplier_point',supplier_point_ref.latitude as 'supplier_lat',supplier_point_ref.longitude as 'supplier_lon'",'link':'supplier_point'}

        query.keys = keys
        query.command = command
        query.joins = joins
        query.wheres = []
        query.havings = []

    def db_rel(query,col_array,filters=False):

        ord_ord = 'join order_details on order_details.OrderID = orders.OrderID'
        pro_ord = 'join products on order_details.ProductID = products.ProductID'
        pro_cat = 'join categories on products.CategoryID = categories.CategoryID'
        pro_sup = 'join suppliers on products.SupplierID = suppliers.SupplierID'
        cus_ord = 'join customers on customers.CustomerID = orders.CustomerID'
        ord_emp = 'join employees on orders.EmployeeID = employees.EmployeeID'
        ord_shi = 'join shippers on orders.ShipperID = shippers.ShipperID'
        cus_iso = "join country_iso as customer_iso_ref on customers.iso_id = customer_iso_ref.country_id"
        sup_iso = "join country_iso as supplier_iso_ref on suppliers.iso_id = supplier_iso_ref.country_id"
        cus_poi = "join city_points as customer_point_ref on customers.point_id = customer_point_ref.city_id"
        sup_poi = "join city_points as supplier_point_ref on suppliers.point_id = supplier_point_ref.city_id"
        indexer = [ord_ord,pro_ord,pro_cat,pro_sup,cus_ord,ord_emp,ord_shi,cus_iso,sup_iso,cus_poi,sup_poi]

        #reference for needed join requests depending its relation to the orders table
        refs = {}
        refs['orders'] = []
        refs['customers'] = [4]
        refs['employees'] = [5]
        refs['shippers'] = [6]
        refs['order_details'] = [0]
        refs['products'] = [0,1]
        refs['suppliers'] = [0,1,3]
        refs['categories'] = [0,1,2]
        refs['customer_iso'] = [4,7]
        refs['supplier_iso'] = [0,1,3,8]
        refs['customer_point'] = [4,9]
        refs['supplier_point'] = [0,1,3,10]

        #take the values from array and parse it with the specified keys
        rels = [query.keys[column] for column in col_array]
        rels_copy = rels.copy()
        
        if filters:
            cat_wheres = "(SELECT orders.OrderID FROM orders {} where {} {} {})"
            cat_haves = """(SELECT orders.OrderID FROM orders 
                JOIN order_details ON order_details.OrderID = orders.OrderID 
                JOIN products ON order_details.ProductID = products.ProductID  
                GROUP BY orders.OrderID HAVING {})"""
            for i in filters:
                is_grouper = i['column'] in ['Total','Quantity']
                command = query.keys[i['column']]['command']
                if ' as ' in command: command = command.split(' as ')[0]
                if not is_grouper:
                    cat_link = refs[query.keys[ i['column']]['link']]
                    cat_link = [indexer[i] for i in cat_link]
                    cat_link = " ".join(cat_link)
                    if isinstance(i['parameter'],str): i['parameter'] = f"'{i['parameter']}'"
                    final_cat = cat_wheres.format(cat_link,command,i['operand'],i['parameter'])
                    query.wheres.append(final_cat)
                elif is_grouper == True:
                    final_group = "SUM({}) {} {}".format(command,i['operand'],i['parameter'])
                    query.havings.append(final_group)

            #query.wheres = " and orders.OrderID IN ".join(query.wheres)
            query.havings = " and ".join(query.havings)
            query.havings = [cat_haves.format(query.havings)] if len(query.havings) > 0 else []
        
        #get the index of sequencial join clauses for relevant columns
        joins = [num for i in rels for num in refs[i['link']]]
        joins = list(set(joins))
        joins.sort()
        joins = [indexer[i] for i in joins]
        joins = " ".join(joins)
        
        #join query string into one
        command = [i['command'] for i in rels_copy]
        command = "orders.OrderID, " + ",".join(command) if len(command) > 0 else "orders.OrderID"

        #add them to the class
        query.command = command
        query.joins = joins

In [399]:
col_array = ['Total', 'SalesPerson']
filters = [{'column': 'SalesPerson', 'parameter': 'Michael Suyama', 'origin': 'Sales Person: Michael Suyama', 'operand': '='}, {'column': 'CategoryName', 'parameter': 'Condiments', 'origin': 'Category Name: Condiments', 'operand': '='}, {'column': 'Total', 'parameter': 100, 'origin': 'Total > 100', 'operand': '>'}, {'column': 'Price', 'parameter': 10, 'origin': 'Price > 10', 'operand': '>'}]

In [410]:
col_array = ['Total', 'CategoryName']
filters = [{'column': 'CategoryName', 'parameter': 'Condiments', 'origin': 'Category Name: Condiments', 'operand': '='}]

In [420]:
col_array = ['Total', 'SalesPerson']
filters = [{'column': 'Total', 'parameter': 1000, 'origin': 'Total > 100', 'operand': '>'},{'column': 'SalesPerson', 'parameter': 'Margaret Peacock', 'origin': 'Sales Person: Michael Suyama', 'operand': '='}]

In [425]:
query = Db_command()
query.db_rel(col_array,filters=False)

In [426]:
test = custom_query(query.command,query.joins,query.wheres,query.havings)

select orders.OrderID, products.Price * order_details.Quantity as 'Total',concat( employees.FirstName,' ',employees.LastName) as 'SalesPerson' from orders join order_details on order_details.OrderID = orders.OrderID join products on order_details.ProductID = products.ProductID join employees on orders.EmployeeID = employees.EmployeeID


In [427]:
test

,OrderID,Total,SalesPerson
0,10258,950.00,Nancy Davolio
1,10258,1387.75,Nancy Davolio
2,10258,192.00,Nancy Davolio
3,10270,570.00,Nancy Davolio
4,10270,1150.00,Nancy Davolio
...,...,...,...
513,10386,67.50,Anne Dodsworth
514,10386,140.00,Anne Dodsworth
515,10411,241.25,Anne Dodsworth
516,10411,778.00,Anne Dodsworth


In [423]:
test.groupby('OrderID').aggregate({'Total':'sum'}).agg(['max','min'])

,Total
max,14104.0
min,1160.1


In [378]:
data = sales()

In [379]:
def sales():
	con.connect()
	select_main = con.cursor()
	select_main.execute('call test_bi()')
	sales = select_main.fetchall()
	con.commit()
	con.close()
	field_names = [i[0] for i in select_main.description]
	sales = pd.DataFrame(sales,columns=field_names)
	sales['OrderID'] = sales['OrderID'].astype(str)
	#sales['OrderDetailID'] = sales['OrderDetailID'].astype(str)
	sales['OrderDate'] = pd.to_datetime(sales['OrderDate'])
	return sales.sort_values('OrderDate')

In [380]:
data = sales()

In [381]:
data =  data[data.columns[~data.columns.str.contains('iso|lat|lon|OrderDetailID')]].sort_values(['OrderDate','OrderID'])

In [396]:
test = data.groupby('OrderID').aggregate({'Total':'sum'}).query('Total>1000').index.values

In [397]:
data[(data.OrderID.isin(test))]

,OrderDate,CustomerName,CustomerCity,CustomerCountry,OrderID,CategoryName,ProductName,Quantity,Price,Total,ShipperName,SupplierName,SupplierCountry,SupplierCity,SalesPerson
415,2019-07-05,Tradição Hipermercados,São Paulo,Brazil,10249,Produce,Manjimup Dried Apples,40,53.00,2120.00,Speedy Express,"G'day, Mate",Australia,Sydney,Michael Suyama
77,2019-07-05,Tradição Hipermercados,São Paulo,Brazil,10249,Produce,Tofu,9,23.25,209.25,Speedy Express,Mayumi's,Japan,Osaka,Michael Suyama
30,2019-07-08,Hanari Carnes,Rio de Janeiro,Brazil,10250,Condiments,Louisiana Fiery Hot Pepper Sauce,15,21.05,315.75,United Package,New Orleans Cajun Delights,USA,New Orleans,Margaret Peacock
416,2019-07-08,Hanari Carnes,Rio de Janeiro,Brazil,10250,Produce,Manjimup Dried Apples,35,53.00,1855.00,United Package,"G'day, Mate",Australia,Sydney,Margaret Peacock
358,2019-07-08,Hanari Carnes,Rio de Janeiro,Brazil,10250,Seafood,Jack's New England Clam Chowder,10,9.65,96.50,United Package,New England Seafood Cannery,USA,Boston,Margaret Peacock
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,2020-02-10,Save-a-lot Markets,Boise,United States of America,10440,Condiments,Sirop d'érable,90,28.50,2565.00,United Package,Forêts d'érables,Canada,Ste-Hyacinthe,Margaret Peacock
191,2020-02-10,Old World Delicatessen,Anchorage,United States of America,10441,Confections,Schoggi Schokolade,50,43.90,2195.00,United Package,Heli Süßwaren GmbH & Co. KG,Germany,Berlin,Janet Leverling
40,2020-02-11,Ernst Handel,Graz,Austria,10442,Condiments,Louisiana Hot Spiced Okra,60,17.00,1020.00,United Package,New Orleans Cajun Delights,USA,New Orleans,Janet Leverling
445,2020-02-11,Ernst Handel,Graz,Austria,10442,Meat/Poultry,Tourtière,80,7.45,596.00,United Package,Ma Maison,Canada,Montréal,Janet Leverling


In [387]:
data.groupby('OrderID').aggregate({'Total':'sum'}).index.values

array(['10248', '10249', '10250', '10251', '10252', '10253', '10254',
       '10255', '10256', '10257', '10258', '10259', '10260', '10261',
       '10262', '10263', '10264', '10265', '10266', '10267', '10268',
       '10269', '10270', '10271', '10272', '10273', '10274', '10275',
       '10276', '10277', '10278', '10279', '10280', '10281', '10282',
       '10283', '10284', '10285', '10286', '10287', '10288', '10289',
       '10290', '10291', '10292', '10293', '10294', '10295', '10296',
       '10297', '10298', '10299', '10300', '10301', '10302', '10303',
       '10304', '10305', '10306', '10307', '10308', '10309', '10310',
       '10311', '10312', '10313', '10314', '10315', '10316', '10317',
       '10318', '10319', '10320', '10321', '10322', '10323', '10324',
       '10325', '10326', '10327', '10328', '10329', '10330', '10331',
       '10332', '10333', '10334', '10335', '10336', '10337', '10338',
       '10339', '10340', '10341', '10342', '10343', '10344', '10345',
       '10346', '103

In [49]:
test1 = data[(data.CategoryName == 'Beverages')].OrderID.values

In [50]:
test2 = data[(data.ProductName == 'Raclette Courdavault')].OrderID.values

In [62]:
test3 = data.groupby('OrderID').aggregate({'Total':'sum'}).query('Total>1000').reset_index().OrderID.values

In [65]:
data[(data.OrderID.isin(list(set.intersection(set(test1),set(test2),set(test3)))))]

,OrderDate,CustomerName,CustomerCity,CustomerCountry,OrderID,CategoryName,ProductName,Quantity,Price,Total,ShipperName,SupplierName,SupplierCountry,SupplierCity,SalesPerson
321,2019-07-12,Richter Supermarkt,Genève,Switzerland,10255,Seafood,Inlagd Sill,25,19.00,475.00,Federal Shipping,Svensk Sjöföda AB,Sweden,Stockholm,Anne Dodsworth
477,2019-07-12,Richter Supermarkt,Genève,Switzerland,10255,Dairy Products,Raclette Courdavault,30,55.00,1650.00,Federal Shipping,Gai pâturage,France,Annecy,Anne Dodsworth
10,2019-07-12,Richter Supermarkt,Genève,Switzerland,10255,Beverages,Chang,20,19.00,380.00,Federal Shipping,Exotic Liquid,UK,Londona,Anne Dodsworth
103,2019-07-12,Richter Supermarkt,Genève,Switzerland,10255,Confections,Pavlova,35,17.45,610.75,Federal Shipping,"Pavlova, Ltd.",Australia,Melbourne,Anne Dodsworth
405,2019-07-29,Frankenversand,München,Germany,10267,Beverages,Lakkalikööri,15,18.00,270.00,Speedy Express,Karkki Oy,Finland,Lappeenranta,Margaret Peacock
478,2019-07-29,Frankenversand,München,Germany,10267,Dairy Products,Raclette Courdavault,70,55.00,3850.00,Speedy Express,Gai pâturage,France,Annecy,Margaret Peacock
350,2019-07-29,Frankenversand,München,Germany,10267,Seafood,Boston Crab Meat,50,18.40,920.00,Speedy Express,New England Seafood Cannery,USA,Boston,Margaret Peacock
506,2019-09-05,Hungry Owl All-Night Grocers,Cork,Ireland,10298,Confections,Tarte au sucre,15,49.30,739.50,United Package,Forêts d'érables,Canada,Ste-Hyacinthe,Michael Suyama
13,2019-09-05,Hungry Owl All-Night Grocers,Cork,Ireland,10298,Beverages,Chang,40,19.00,760.00,United Package,Exotic Liquid,UK,Londona,Michael Suyama
323,2019-09-05,Hungry Owl All-Night Grocers,Cork,Ireland,10298,Seafood,Inlagd Sill,40,19.00,760.00,United Package,Svensk Sjöföda AB,Sweden,Stockholm,Michael Suyama


In [39]:
data[(data.OrderID == '10267')]

,OrderDate,CustomerName,CustomerCity,CustomerCountry,OrderID,CategoryName,ProductName,Quantity,Price,Total,ShipperName,SupplierName,SupplierCountry,SupplierCity,SalesPerson
405,2019-07-29,Frankenversand,München,Germany,10267,Beverages,Lakkalikööri,15,18.0,270.0,Speedy Express,Karkki Oy,Finland,Lappeenranta,Margaret Peacock
478,2019-07-29,Frankenversand,München,Germany,10267,Dairy Products,Raclette Courdavault,70,55.0,3850.0,Speedy Express,Gai pâturage,France,Annecy,Margaret Peacock
350,2019-07-29,Frankenversand,München,Germany,10267,Seafood,Boston Crab Meat,50,18.4,920.0,Speedy Express,New England Seafood Cannery,USA,Boston,Margaret Peacock


In [25]:
data[(data.OrderID == '10335')][['CategoryName','ProductName','SupplierName','SupplierCountry']]

,CategoryName,ProductName,SupplierName,SupplierCountry
418,Produce,Manjimup Dried Apples,"G'day, Mate",Australia
246,Dairy Products,Gorgonzola Telino,Formaggi Fortini s.r.l.,Italy
258,Dairy Products,Mascarpone Fabioli,Formaggi Fortini s.r.l.,Italy
15,Beverages,Chang,Exotic Liquid,UK


In [6]:
data =  data[data.columns[~data.columns.str.contains('iso|lat|lon|OrderDetailID')]].sort_values(['OrderDate','OrderID'])

In [7]:
data

,OrderDate,CustomerName,CustomerCity,CustomerCountry,OrderID,CategoryName,ProductName,Quantity,Price,Total,ShipperName,SupplierName,SupplierCountry,SupplierCity,SalesPerson
59,2019-07-04,Wilman Kala,Helsinki,Finland,10248,Dairy Products,Queso Cabrales,12,21.0,252.0,Federal Shipping,Cooperativa de Quesos 'Las Cabras',Spain,Oviedo,Steven Buchanan
367,2019-07-04,Wilman Kala,Helsinki,Finland,10248,Grains/Cereals,Singaporean Hokkien Fried Mee,10,14.0,140.0,Federal Shipping,Leka Trading,Singapore,Singapore,Steven Buchanan
259,2019-07-04,Wilman Kala,Helsinki,Finland,10248,Dairy Products,Mozzarella di Giovanni,5,34.8,174.0,Federal Shipping,Formaggi Fortini s.r.l.,Italy,Ravenna,Steven Buchanan
415,2019-07-05,Tradição Hipermercados,São Paulo,Brazil,10249,Produce,Manjimup Dried Apples,40,53.0,2120.0,Speedy Express,"G'day, Mate",Australia,Sydney,Michael Suyama
77,2019-07-05,Tradição Hipermercados,São Paulo,Brazil,10249,Produce,Tofu,9,23.25,209.25,Speedy Express,Mayumi's,Japan,Osaka,Michael Suyama
30,2019-07-08,Hanari Carnes,Rio de Janeiro,Brazil,10250,Condiments,Louisiana Fiery Hot Pepper Sauce,15,21.05,315.75,United Package,New Orleans Cajun Delights,USA,New Orleans,Margaret Peacock
416,2019-07-08,Hanari Carnes,Rio de Janeiro,Brazil,10250,Produce,Manjimup Dried Apples,35,53.0,1855.0,United Package,"G'day, Mate",Australia,Sydney,Margaret Peacock
358,2019-07-08,Hanari Carnes,Rio de Janeiro,Brazil,10250,Seafood,Jack's New England Clam Chowder,10,9.65,96.5,United Package,New England Seafood Cannery,USA,Boston,Margaret Peacock
465,2019-07-08,Victuailles en stock,Lyon,France,10251,Grains/Cereals,Ravioli Angelo,15,19.5,292.5,Speedy Express,Pasta Buttini s.r.l.,Italy,Salerno,Janet Leverling
31,2019-07-08,Victuailles en stock,Lyon,France,10251,Condiments,Louisiana Fiery Hot Pepper Sauce,20,21.05,421.0,Speedy Express,New Orleans Cajun Delights,USA,New Orleans,Janet Leverling


In [9]:
test = []
customers = data.CustomerName.tolist()
for dat in data:
    selected = data[dat]
    new_data = {'column':dat,'values':[]}
    for num,value in enumerate(selected.values):
        if len(new_data['values'])> 0 and value == new_data['values'][-1]['name'] and customers[num-1] == customers[num]:
            new_data['values'][-1]['span']  += 1
        else:
            new_data['values'].append({'name':str(value),'span':1,'index':num})
    test.append(new_data)

In [10]:
data

,OrderDate,CustomerName,CustomerCity,CustomerCountry,OrderID,CategoryName,ProductName,Quantity,Price,Total,ShipperName,SupplierName,SupplierCountry,SupplierCity,SalesPerson
59,2019-07-04,Wilman Kala,Helsinki,Finland,10248,Dairy Products,Queso Cabrales,12,21.0,252.0,Federal Shipping,Cooperativa de Quesos 'Las Cabras',Spain,Oviedo,Steven Buchanan
367,2019-07-04,Wilman Kala,Helsinki,Finland,10248,Grains/Cereals,Singaporean Hokkien Fried Mee,10,14.0,140.0,Federal Shipping,Leka Trading,Singapore,Singapore,Steven Buchanan
259,2019-07-04,Wilman Kala,Helsinki,Finland,10248,Dairy Products,Mozzarella di Giovanni,5,34.8,174.0,Federal Shipping,Formaggi Fortini s.r.l.,Italy,Ravenna,Steven Buchanan
415,2019-07-05,Tradição Hipermercados,São Paulo,Brazil,10249,Produce,Manjimup Dried Apples,40,53.0,2120.0,Speedy Express,"G'day, Mate",Australia,Sydney,Michael Suyama
77,2019-07-05,Tradição Hipermercados,São Paulo,Brazil,10249,Produce,Tofu,9,23.25,209.25,Speedy Express,Mayumi's,Japan,Osaka,Michael Suyama
30,2019-07-08,Hanari Carnes,Rio de Janeiro,Brazil,10250,Condiments,Louisiana Fiery Hot Pepper Sauce,15,21.05,315.75,United Package,New Orleans Cajun Delights,USA,New Orleans,Margaret Peacock
416,2019-07-08,Hanari Carnes,Rio de Janeiro,Brazil,10250,Produce,Manjimup Dried Apples,35,53.0,1855.0,United Package,"G'day, Mate",Australia,Sydney,Margaret Peacock
358,2019-07-08,Hanari Carnes,Rio de Janeiro,Brazil,10250,Seafood,Jack's New England Clam Chowder,10,9.65,96.5,United Package,New England Seafood Cannery,USA,Boston,Margaret Peacock
465,2019-07-08,Victuailles en stock,Lyon,France,10251,Grains/Cereals,Ravioli Angelo,15,19.5,292.5,Speedy Express,Pasta Buttini s.r.l.,Italy,Salerno,Janet Leverling
31,2019-07-08,Victuailles en stock,Lyon,France,10251,Condiments,Louisiana Fiery Hot Pepper Sauce,20,21.05,421.0,Speedy Express,New Orleans Cajun Delights,USA,New Orleans,Janet Leverling


In [11]:
stuff = {'CustomerName': {'count': 74}, 'CustomerCity': {'count': 58}, 'CustomerCountry': {'count': 21}, 'OrderID': {'count': 196}, 'CategoryName': {'count': 8}, 'ProductName': {'count': 77}, 'ShipperName': {'count': 3}, 'SupplierName': {'count': 29}, 'SupplierCountry': {'count': 16}, 'SupplierCity': {'count': 29}, 'SalesPerson': {'count': 9}, 'Quantity': {'max': 241, 'min': 2}, 'Price': {'max': 263.5, 'min': 2.5}, 'Total': {'max': 15353.6, 'min': 60.0}, 'OrderDate': {'max': '2020-02-12', 'min': '2019-07-04'}}


In [15]:
test = data.columns.tolist()

In [17]:
[{i:stuff[i]} for i in test]

[{'OrderDate': {'max': '2020-02-12', 'min': '2019-07-04'}},
 {'CustomerName': {'count': 74}},
 {'CustomerCity': {'count': 58}},
 {'CustomerCountry': {'count': 21}},
 {'OrderID': {'count': 196}},
 {'CategoryName': {'count': 8}},
 {'ProductName': {'count': 77}},
 {'Quantity': {'max': 241, 'min': 2}},
 {'Price': {'max': 263.5, 'min': 2.5}},
 {'Total': {'max': 15353.6, 'min': 60.0}},
 {'ShipperName': {'count': 3}},
 {'SupplierName': {'count': 29}},
 {'SupplierCountry': {'count': 16}},
 {'SupplierCity': {'count': 29}},
 {'SalesPerson': {'count': 9}}]

In [13]:
[{key:value} for key,value in stuff.items()]

[{'CustomerName': {'count': 74}},
 {'CustomerCity': {'count': 58}},
 {'CustomerCountry': {'count': 21}},
 {'OrderID': {'count': 196}},
 {'CategoryName': {'count': 8}},
 {'ProductName': {'count': 77}},
 {'ShipperName': {'count': 3}},
 {'SupplierName': {'count': 29}},
 {'SupplierCountry': {'count': 16}},
 {'SupplierCity': {'count': 29}},
 {'SalesPerson': {'count': 9}},
 {'Quantity': {'max': 241, 'min': 2}},
 {'Price': {'max': 263.5, 'min': 2.5}},
 {'Total': {'max': 15353.6, 'min': 60.0}},
 {'OrderDate': {'max': '2020-02-12', 'min': '2019-07-04'}}]